In [58]:
import pandas as pd
from clickhouse_driver import Client
  
client = Client(host='clickhouse.sofascore.ai',
                port='9000',
                user='lkovacevic',
                password='b9Ea8XzdFr0w6Zo32i4Z'
)

create_table="""CREATE TABLE IF NOT EXISTS lkovacevic.l2_dataset
(
eventDate String,
eventName String,
userPseudoId String,
platform LowCardinality(String),
status LowCardinality(String),
geoCountry LowCardinality(String),
id Nullable(Int64)
)
ENGINE = MergeTree()
ORDER BY eventDate
SETTINGS index_granularity = 8192;"""

client.execute(create_table)

chunksize = 1000000
file_path = '/Users/lucij/Desktop/processed_data.csv'
na_values = [r'\N', '','NaN'] 

dtype_mapping = {
    'eventDate': 'str',  
    'eventName': 'str',
    'userPseudoId': 'str',
    'platform': 'str',
    'status': 'str',
    'geoCountry': 'str',
    'id': 'str', 
}
for chunk_df in pd.read_csv(file_path, chunksize=chunksize,na_values=na_values,dtype=dtype_mapping):
    chunk_df['id'] = pd.to_numeric(chunk_df['id'], errors='coerce')
    chunk_df['id'] = chunk_df['id'].where(pd.notna(chunk_df['id']), None)
    chunk_df = chunk_df.where(pd.notna(chunk_df), None)
    
    client.insert_dataframe('INSERT INTO lkovacevic.l2_dataset(eventDate, eventName, userPseudoId, platform, status, geoCountry, id) VALUES', chunk_df, settings=dict(use_numpy=True))
  


  eventDate    eventName                      userPseudoId geoCountry  \
0  20230301  open_league  234EA969F32E42B89DCCE7C093DD0741      Italy   

  platform     id status  
0      IOS  11446    NaN  
        eventDate  eventName                      userPseudoId geoCountry  \
1000000  20230305  open_team  815CB3593DF940B4A1942B82625D6AB0     Serbia   

        platform    id status  
1000000      IOS  3580    NaN  
        eventDate   eventName                      userPseudoId geoCountry  \
2000000  20230305  open_event  a1690436569f98770fad82fdd75792aa      Italy   

        platform        id       status  
2000000  ANDROID  10411702  In progress  
        eventDate  eventName                      userPseudoId geoCountry  \
3000000  20230307  open_team  683c7c5dbf2fcdebb2dd2f0ac3999730    Germany   

        platform     id status  
3000000  ANDROID  34416    NaN  
        eventDate   eventName                      userPseudoId geoCountry  \
4000000  20230308  open_event  3A831E0D7